Evaluation

Import environment variables

In [11]:
import dotenv
import os

dotenv.load_dotenv()


True

Load docs

In [107]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "test_data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

docs[0]

Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2023-07-15T11:40:07+05:30', 'source': 'test_data/INTRODUCTION-TO-RDBMS.pdf', 'file_path': 'test_data/INTRODUCTION-TO-RDBMS.pdf', 'total_pages': 24, 'format': 'PDF 1.7', 'title': 'Microsoft Word - Introduction to relational databases', 'author': 'Administrator', 'subject': '', 'keywords': '', 'moddate': '2023-07-15T11:40:07+05:30', 'trapped': '', 'modDate': "D:20230715114007+05'30'", 'creationDate': "D:20230715114007+05'30'", 'page': 0}, page_content='Introduction to relational databases \n \nIntroduction to relational databases \n \n\uf0b7 A relational database is based on the relational model and uses a collection of \ntables to represent both data and the relationship among those data. \n\uf0b7 It also includes DDL and DML  \n\uf0b7 Relational database was originally defined by Edger Codd at IBM Research \ncenter in 1970. \n\uf0b7 In relational database, user only needs to understand logical str

choose Llm

In [108]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.embeddings import OpenAIEmbeddings
import openai

evaluator_llm = evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
openai_client = openai.OpenAI()
generator_embeddings = OpenAIEmbeddings(client=openai_client)

/tmp/ipykernel_82574/2774485547.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
/tmp/ipykernel_82574/2774485547.py:7: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))


Generate testset

In [109]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=5)

Applying SummaryExtractor:   0%|          | 0/23 [00:00<?, ?it/s]

Applying EmbeddingExtractor:   0%|          | 0/23 [00:00<?, ?it/s]/root/py/proj/backend/.venv/lib/python3.13/site-packages/ragas/testset/transforms/base.py:188: UserWarning: Using sync embedding model OpenAIEmbeddings in async context. This may impact performance. Consider using an async-compatible embedding model for better performance.
  property_name, property_value = await self.extract(node)
Generating Samples: 100%|██████████| 6/6 [00:06<00:00,  1.06s/it]


In [110]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Who defined the relational database model?,[Introduction to relational databases \n \nInt...,The relational database model was originally d...,single_hop_specific_query_synthesizer
1,What role does a DBA play in creating stored p...,[ A stored procedure is a high-end database t...,A DBA will often create stored procedures to h...,single_hop_specific_query_synthesizer
2,How do applications utilize the ODBC API to co...,[<1-hop>\n\n Applications such as \no Graphic...,"Applications such as graphical interfaces, spr...",multi_hop_abstract_query_synthesizer
3,How does the full outer join operation relate ...,[<1-hop>\n\nName \nTamil \nSelva \nDurai \nKut...,The full outer join operation in SQL takes all...,multi_hop_abstract_query_synthesizer
4,What is the difference between static and dyna...,[<1-hop>\n\n Dynamic SQL is a programming tec...,The difference between static and dynamic SQL ...,multi_hop_specific_query_synthesizer
5,What titles of books can be displayed using th...,[<1-hop>\n\n3 \nPYTHON \n450 \n2009 \n4 \nDS \...,The titles of books that can be displayed usin...,multi_hop_specific_query_synthesizer


Prepere data

In [111]:
from agent.agent import tutor_agent
from langchain_core.messages import HumanMessage
import json


In [102]:
#THIS CELL IS FOR DEBUGGING

# from agent.utils.tools import read_from_temp

# eval_contexts = read_from_temp()
# eval_contexts

import json

res = tutor_agent.invoke({"messages": [HumanMessage("How to use key?")], "retrieved_docs": []})
msgs = res["messages"]

retrieved_contexts = []

for msg in msgs:
    if msg.name == "knowledge_base_search":
        retrieved_contexts.append(msg.content)

parsed_contents = [json.loads(item) for item in retrieved_contexts]

contents = []
for inner_list in parsed_contents:
    for item in inner_list:
        contents.append(item)

contents

# res["retrieved_docs"]


['7.6 Keys \n \nYou might wish to read again the material under the heading Keys in Chapter 6, Section 6.4, which gives \nthe following definitions: \n \nDefinitions for superkey and key',
 'KeyNumber  INTEGER, \n                                     KeyAttr    NAME } \n                               KEY { ALL BUT } ;',
 'definition of “key” can be generalized to apply to relational expressions as well as to relvars. For example,',
 'Chapter 6 draws attention to two special cases of keys: the entire heading as a key, and the empty set as a',
 'key can possibly exist for the relvar it applies to. Exercise for the reader: Why is this so?']

In [112]:
for test_row in dataset:
    query = test_row.eval_sample.user_input
    response = tutor_agent.invoke({"messages": [HumanMessage(query)]})

    msgs = response["messages"]

    retrieved_contexts = []

    for msg in msgs:
        if msg.name == "knowledge_base_search":
            retrieved_contexts.append(msg.content)

    parsed_contents = [json.loads(item) for item in retrieved_contexts]

    contents = []
    for inner_list in parsed_contents:
        for item in inner_list:
            contents.append(item)

    test_row.eval_sample.retrieved_contexts = contents
    test_row.eval_sample.response = response["messages"][-1].content
    

In [113]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

Run evaluation

In [114]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

Evaluating: 100%|██████████| 18/18 [00:34<00:00,  1.92s/it]


{'context_recall': 0.9444, 'faithfulness': 0.7792, 'factual_correctness(mode=f1)': 0.7250}